In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data2014 = pd.read_csv("../Week7/Input/2014dataset.csv",low_memory=False)
data2015 = pd.read_csv("../Week7/Input/2015dataset.csv",low_memory=False)
data2016 = pd.read_csv("../Week7/Input/2016dataset.csv",low_memory=False)

In [3]:
important_fields = ["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME", "STATE", "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
                    "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)", 
                     "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)", "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                    "7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15", "1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15",
                    "2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15"]

data2014 = data2014[important_fields].dropna(how='all')
data2015 = data2015[important_fields].dropna(how='all')
data2016 = data2016[important_fields].dropna(how='all')

def add_columns(playground1):
    playground1["IndPremiumPMM"] = playground1["1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndCostsPMM"] = playground1["2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndRTPMM"] = playground1["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndPremiumMinusCostsPMM"] = playground1["IndPremiumPMM"] - playground1["IndCostsPMM"]

add_columns(data2014)
add_columns(data2015)
add_columns(data2016)

In [5]:
merged2 = data2014.merge(data2015, on='HIOS ID', how='outer', suffixes=('_2014', '_2015'))
merged2 = merged2.where(merged2["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014"] > 2000).dropna(how='all')
merged2 = merged2.where(merged2["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2015"] > 2000).dropna(how='all')
merged3 = merged2.merge(data2016, on='HIOS ID', how='outer', suffixes=('', '_2016'))
#merged3 = merged3.where(merged3["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"] > 2000).dropna(how='all')
merged3.to_csv("MarketExitAll_no2000.csv")

In [6]:
def unweightedr(dataset, column_name1, column_name2):
    if column_name1 == column_name2: # when the two columns are the same...
        dataset = dataset[[column_name1]].dropna(how='any')
        n = len(dataset.index)
        sum_b = sum(dataset[column_name1])
        avg_b = sum_b / n
    else: # when the two columns are different
        dataset = dataset[[column_name1, column_name2]].dropna(how='any')
        n = len(dataset.index)
        sum_b = sum(dataset[column_name2])
        avg_b = sum_b / n
    sum_a = sum(dataset[column_name1])
    avg_a = sum_a / n
    print("Average " + str(column_name1) + " : " + str(avg_a) + "; Average " + str(column_name2) + " : " + str(avg_b) + " with n = " + str(n))
    num = 0
    var_a = 0
    var_b = 0
    for index, row in dataset.iterrows():
        a_i = row[0]
        b_i = row[0]
        if column_name1 != column_name2:
            b_i = row[1]
        num = num + (a_i - avg_a)*(b_i - avg_b)
        var_a = var_a + (a_i - avg_a)**2
        var_b = var_b + (b_i - avg_b)**2   
        index = index + 1
    std_a = np.sqrt(var_a)
    std_b = np.sqrt(var_b)
    denom = std_a * std_b
    print("r for " + str(column_name1) + " vs " + str(column_name2) + " = " + str(num/denom) + " with n = " + str(n))
    return num / denom

In [7]:
df = pd.read_csv("MarketExitAll_filtered.csv")

In [8]:
df["MMChange1615"] = (df["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
                      - df["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2015"])
df["MMChange1514"] = (df["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2015"]
                      - df["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15_2014"])
df["PremChange1615"] = df["IndPremiumPMM"] - df["IndPremiumPMM_2015"]
df["PremChange1514"] = df["IndPremiumPMM_2015"] - df["IndPremiumPMM_2014"]

In [9]:
unweightedr(df, "IndPremiumMinusCostsPMM_2014", "MMChange1514")
unweightedr(df, "IndPremiumMinusCostsPMM_2014", "MMChange1615")
unweightedr(df, "IndPremiumMinusCostsPMM_2015", "MMChange1615")
unweightedr(df, "IndPremiumMinusCostsPMM_2014", "PremChange1514")
unweightedr(df, "IndPremiumMinusCostsPMM_2014", "PremChange1615")
unweightedr(df, "IndPremiumMinusCostsPMM_2015", "PremChange1615")

Average IndPremiumMinusCostsPMM_2014 : -52.8101675978; Average MMChange1514 : 75797.7851459 with n = 377
r for IndPremiumMinusCostsPMM_2014 vs MMChange1514 = 0.000132191159748 with n = 377
Average IndPremiumMinusCostsPMM_2014 : -46.213948329; Average MMChange1615 : -10520.0294118 with n = 306
r for IndPremiumMinusCostsPMM_2014 vs MMChange1615 = 0.0388734080478 with n = 306
Average IndPremiumMinusCostsPMM_2015 : -48.2936827318; Average MMChange1615 : -10520.0294118 with n = 306
r for IndPremiumMinusCostsPMM_2015 vs MMChange1615 = 0.085465752061 with n = 306
Average IndPremiumMinusCostsPMM_2014 : -52.8101675978; Average PremChange1514 : 29.6259295532 with n = 377
r for IndPremiumMinusCostsPMM_2014 vs PremChange1514 = -0.005517438097 with n = 377
Average IndPremiumMinusCostsPMM_2014 : -46.213948329; Average PremChange1615 : 31.2530796768 with n = 306
r for IndPremiumMinusCostsPMM_2014 vs PremChange1615 = 0.0367182585727 with n = 306
Average IndPremiumMinusCostsPMM_2015 : -48.2936827318; A

-0.075541662825284031

In [12]:
unweightedr(df, "IndCostsPMM_2014", "IndPremiumPMM_2014")

Average IndCostsPMM_2014 : 367.472352174; Average IndPremiumPMM_2014 : 314.662184572 with n = 377
r for IndCostsPMM_2014 vs IndPremiumPMM_2014 = 0.644176098265 with n = 377


0.64417609826465871